In [1]:
# Import Required Libraries

import numpy as np
import pandas as pd
import json
from sklearn.metrics import mean_squared_error,mean_absolute_error

### 1: Extract and Read Dataset

In [2]:
# Extracting text

from striprtf.striprtf import rtf_to_text

with open("algoparams_from_ui.json.rtf") as infile:
    content = infile.read()
    text = rtf_to_text(content)
print(text)  

{
    "session_name": "test",
    "session_description": "test",
    "design_state_data": {

      "session_info" : {
        "project_id": "1",
        "experiment_id": "kkkk-11",
        "dataset":"iris_modified.csv",
        "session_name": "test",
        "session_description": "test"
        },

      "target": {
        "prediction_type": "Regression",
        "target": "petal_width",
        "type":"regression",
        "partitioning": true
      },
      "train": {
        "policy": "Split the dataset",
        "time_variable": "sepal_length",
        "sampling_method": "No sampling(whole data)",
        "split": "Randomly",
        "k_fold": false,
        "train_ratio": 0,
        "random_seed": 0
      },
      "metrics": {
        "optomize_model_hyperparameters_for": "AUC",
        "optimize_threshold_for": "F1 Score",
        "compute_lift_at": 0,
        "cost_matrix_gain_for_true_prediction_true_result": 1,
        "cost_matrix_gain_for_true_prediction_false_result": 0,

In [3]:
type(text)

str

In [4]:

data_UI= json.loads(text)

In [5]:
type(data_UI)

dict

### 2: Reading Features & Feature Handling

In [6]:
df = pd.read_csv(data_UI['design_state_data']['session_info']['dataset'])

In [7]:
Target = data_UI['design_state_data']['target']
Models = data_UI['design_state_data']['algorithms']

In [8]:
Target

{'prediction_type': 'Regression',
 'target': 'petal_width',
 'type': 'regression',
 'partitioning': True}

In [9]:
Models

{'RandomForestClassifier': {'model_name': 'Random Forest Classifier',
  'is_selected': False,
  'min_trees': 10,
  'max_trees': 30,
  'feature_sampling_statergy': 'Default',
  'min_depth': 20,
  'max_depth': 30,
  'min_samples_per_leaf_min_value': 5,
  'min_samples_per_leaf_max_value': 50,
  'parallelism': 0},
 'RandomForestRegressor': {'model_name': 'Random Forest Regressor',
  'is_selected': True,
  'min_trees': 10,
  'max_trees': 20,
  'feature_sampling_statergy': 'Default',
  'min_depth': 20,
  'max_depth': 25,
  'min_samples_per_leaf_min_value': 5,
  'min_samples_per_leaf_max_value': 10,
  'parallelism': 0},
 'GBTClassifier': {'model_name': 'Gradient Boosted Trees',
  'is_selected': False,
  'num_of_BoostingStages': [67, 89],
  'feature_sampling_statergy': 'Fixed number',
  'learningRate': [],
  'use_deviance': True,
  'use_exponential': False,
  'fixed_number': 22,
  'min_subsample': 1,
  'max_subsample': 2,
  'min_stepsize': 0.1,
  'max_stepsize': 0.5,
  'min_iter': 20,
  'max_i

#### EDA

In [10]:
#read csv file

data = pd.read_csv('iris.csv')

In [11]:
data.shape

(150, 5)

In [12]:
#Checking for null values are present or not
data.isnull().sum()

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64

In [13]:
data['species'].value_counts()


Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: species, dtype: int64

In [14]:
data.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [16]:
# Encoding the dataset

data = pd.get_dummies(data,columns=['species'],drop_first = True)

In [17]:
X = data.drop(columns = ['petal_width'],axis = 1)
Y = data.petal_width

### 3: Building Models

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=2)

In [19]:
#Using Random Forest Regression

from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(n_estimators = 20 ,max_depth = 20 , min_samples_leaf = 10)
rfr.fit(X_train,Y_train)
print('Random Forest Regression Training Accuracy:', rfr.score(X_train, Y_train)*100) 

Random Forest Regression Training Accuracy: 95.54365842528117


In [20]:
pred = rfr.predict(X_test)
mse = mean_squared_error(Y_test,pred)
rmse = mean_squared_error(Y_test,pred,squared=False)
mae = mean_absolute_error(Y_test,pred)
print(f"Mean Squared Error for Random_Forest_Regressor :  {mse}")
print(f"Root Mean Squared Error Random_Fores_tRegressor : {rmse}")
print(f"Mean Absolute Error for Random_Forest_Regressor :{mae}")

Mean Squared Error for Random_Forest_Regressor :  0.03119322441094824
Root Mean Squared Error Random_Fores_tRegressor : 0.17661603667546227
Mean Absolute Error for Random_Forest_Regressor :0.13131017561437552


In [21]:
#Using Linear Regression
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train,Y_train)
print('Linear Regression Training Accuracy:', lr.score(X_train, Y_train)*100)

Linear Regression Training Accuracy: 95.37415449043009


In [22]:
pred = lr.predict(X_test)
mse = mean_squared_error(Y_test,pred)
rmse = mean_squared_error(Y_test,pred,squared=False)
mae = mean_absolute_error(Y_test,pred)
print(f"Mean Squared Error for Linear_Regression :  {mse}")
print(f"Root Mean Squared Error Linear_Regression : {rmse}")
print(f"Mean Absolute Error for Linear_Regression : {mae}")

Mean Squared Error for Linear_Regression :  0.02940100195363566
Root Mean Squared Error Linear_Regression : 0.17146720372606436
Mean Absolute Error for Linear_Regression : 0.12488333462996462


In [23]:
#Using Gradient Boosting Regressor
from sklearn.ensemble import GradientBoostingRegressor
gbtr = GradientBoostingRegressor(n_estimators= 89,max_depth=7)
gbtr.fit(X_train,Y_train)
print('Gradient Boosting Regressor Training Accuracy:', gbtr.score(X_train, Y_train)*100)  

Gradient Boosting Regressor Training Accuracy: 99.99244317158403


In [24]:
pred = gbtr.predict(X_test)
mse = mean_squared_error(Y_test,pred)
rmse = mean_squared_error(Y_test,pred,squared=False)
mae = mean_absolute_error(Y_test,pred)
print(f"Mean Squared Error for Gradient_Boosting_Regressor :  {mse}")
print(f"Root Mean Squared Error Gradient_Boosting_Regressor : {rmse}")
print(f"Mean Absolute Error for Gradient_Boosting_Regressor : {mae}")

Mean Squared Error for Gradient_Boosting_Regressor :  0.0490859163852967
Root Mean Squared Error Gradient_Boosting_Regressor : 0.22155341655071967
Mean Absolute Error for Gradient_Boosting_Regressor : 0.1487461422866403


In [25]:
#Using Lasso Regression
from sklearn.linear_model import Lasso
lsr = Lasso(max_iter = 50)
lsr.fit(X_train,Y_train)
print('Lasso Regression Training Accuracy:', lsr.score(X_train, Y_train)*100)

Lasso Regression Training Accuracy: 33.16588829512976


In [26]:
pred = lsr.predict(X_test)
mse = mean_squared_error(Y_test,pred)
rmse = mean_squared_error(Y_test,pred,squared=False)
mae = mean_absolute_error(Y_test,pred)
print(f"Mean Squared Error for Lasso Regression : {mse}")
print(f"Root Mean Squared Error Lasso Regression : {rmse}")
print(f"Mean Absolute Error for Lasso Regression :{mae}")

Mean Squared Error for Lasso Regression : 0.43390883616450243
Root Mean Squared Error Lasso Regression : 0.6587175693455447
Mean Absolute Error for Lasso Regression :0.5706362531691952


In [27]:
#Using Xgboost Regression
import xgboost as xg
xgb = xg.XGBRegressor()
xgb.fit(X_train,Y_train)
print('XGBoost Regression Training Accuracy:', xgb.score(X_train, Y_train)*100)

XGBoost Regression Training Accuracy: 99.9909468706498


In [28]:
pred = xgb.predict(X_test)
mse = mean_squared_error(Y_test,pred)
rmse = mean_squared_error(Y_test,pred,squared=False)
mae = mean_absolute_error(Y_test,pred)
print(f"Mean Squared Error for Xgboost_Regression : {mse}")
print(f"Root Mean Squared Error Xgboost_Regression : {rmse}")
print(f"Mean Absolute Error for Xgboost_Regression :{mae}")

Mean Squared Error for Xgboost_Regression : 0.04638244483512975
Root Mean Squared Error Xgboost_Regression : 0.2153658395269077
Mean Absolute Error for Xgboost_Regression :0.14557498544454575


In [29]:
#Using Decision Tree regression
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor(min_samples_leaf = 12)
dtr.fit(X_train,Y_train)
print('Decision Tree Regression Training Accuracy:', dtr.score(X_train, Y_train)*100)

Decision Tree Regression Training Accuracy: 94.89157167973767


In [30]:
pred = dtr.predict(X_test)
mse = mean_squared_error(Y_test,pred)
rmse = mean_squared_error(Y_test,pred,squared=False)
mae = mean_absolute_error(Y_test,pred)
print(f"Mean Squared Error for  Decision_Tree_Regressor : {mse}")
print(f"Root Mean Squared Error  Decision_Tree_Regressor : {rmse}")
print(f"Mean Absolute Error for  Decision_Tree_Regressor : {mae}")

Mean Squared Error for  Decision_Tree_Regressor : 0.03354250257201645
Root Mean Squared Error  Decision_Tree_Regressor : 0.18314612355170515
Mean Absolute Error for  Decision_Tree_Regressor : 0.13384259259259254


In [31]:
#Using SVR Regression
from sklearn.svm import SVR
svr = SVR(kernel = 'rbf',gamma = 7,max_iter = 7)
svr.fit(X_train,Y_train)
print('SVM Regression Training Accuracy:', svr.score(X_train, Y_train)*100)

SVM Regression Training Accuracy: 85.30640483214395


C:\Users\DELL\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=7).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


In [32]:
pred = svr.predict(X_test)
mse = mean_squared_error(Y_test,pred)
rmse = mean_squared_error(Y_test,pred,squared=False)
mae = mean_absolute_error(Y_test,pred)
print(f"Mean Squared Error for SVR Regression : {mse}")
print(f"Root Mean Squared Error SVR Regression : {rmse}")
print(f"Mean Absolute Error for SVR Regressionr : {mae}")

Mean Squared Error for SVR Regression : 0.15438432323287438
Root Mean Squared Error SVR Regression : 0.3929177054204536
Mean Absolute Error for SVR Regressionr : 0.3020351226837967


### 4: Outcome

**Thus we can conclude that:**

     Model with highest training accuracy: XGboost Regression & Gradient Boosting Regressor (99.99%)
     Model with lowest RMSE: Linear Regressor & Random Forest (0.17)